In [26]:
#Import des csv

import pandas as pd
import matplotlib.pyplot as pyplot
import datetime as dt
import statsmodels.api as sm


On va d'abord traite les données afin de pouvoir les utiliser correctement par la suite.

In [13]:
crypto_prices = pd.read_csv("crypto_prices_v2.csv")
# Convert timestamp column to datetime format
crypto_prices['timestamp'] = pd.to_datetime(crypto_prices['timestamp'])

# Set timestamp column as index
crypto_prices.set_index('timestamp', inplace=True)
crypto_prices.head()

,unfi-busd,iota-usd,bhp-usdt,dot-usdt,aave-usdt,nmc-usdt,mx-usdt,dht-usdt,crv-usd,pha-usdt,...,api3-usdt,1inch-usdt,comp-usdt,ersdl-usdt,xor-usdt,ren-usdt,bid-usdt,mkr-usdt,prq-usdt,mta-usdt
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01,5.562509,0.294552,0.664400,8.798804,87.590000,0.439790,0.1436,0.859310,0.637000,0.118509,...,1.801849,1.167545,144.668659,0.008365,102.525652,0.327979,0.013622,589.914791,0.527962,0.909049
2021-01-02,5.221067,0.286993,0.667342,8.760201,85.937751,0.459137,0.1452,0.992900,0.592534,0.117040,...,1.834821,1.090871,140.000168,0.008287,98.931513,0.312595,0.013083,597.494350,0.460791,1.011624
2021-01-03,6.482086,0.298317,0.667600,9.279673,87.968375,0.507415,0.1388,0.959000,0.621000,0.115557,...,1.889542,1.084320,145.591570,0.009397,101.388139,0.325757,0.014231,672.198175,0.507648,1.160168
2021-01-04,5.851341,0.317800,0.660000,9.465044,102.336182,0.521261,0.1384,1.057248,0.672000,0.114000,...,2.175675,1.076138,145.562839,0.012118,110.576618,0.329471,0.013833,678.422680,0.451232,1.165947
2021-01-05,5.751700,0.354807,0.652600,9.612824,114.077356,0.510710,0.1376,1.171523,0.680000,0.115772,...,2.388763,1.141967,155.225308,0.012469,115.365119,0.335810,0.012865,705.739433,0.608839,1.199186


In [3]:
exchange_info = pd.read_csv("exchange_info_v2.csv")
exchange_info.head()

,Unnamed: 0,CeX,DeX
0,unfi-busd,1,0
1,iota-usd,1,0
2,bhp-usdt,1,0
3,dot-usdt,1,0
4,aave-usdt,1,0


In [51]:
crix = pd.read_csv("crix.csv", delimiter=";")
crix['date'] = pd.to_datetime(crix['date'])
crix.head()


C:\Users\alanm\AppData\Local\Temp\ipykernel_28364\4172023063.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  crix['date'] = pd.to_datetime(crix['date'])


,date,price
0,2014-07-31,1000
1,2014-08-01,"1018,202717"
2,2014-08-02,"1008,772389"
3,2014-08-03,"1004,4165"
4,2014-08-04,"1004,984138"


In [50]:
data_factors = pd.read_csv("data_factors.csv", delimiter=";", header=4)
data_factors = data_factors.rename(columns={data_factors.columns[0]: 'timestamp'})
data_factors = data_factors.iloc[:-1]
# Convert timestamp column to datetime format
data_factors['timestamp'] = pd.to_datetime(data_factors['timestamp'])
data_factors.head()




,timestamp,Mkt-RF,SMB,HML,RF
0,1926-07-01,"0,1","-0,25","-0,27","0,009"
1,1926-07-02,"0,45","-0,33","-0,06","0,009"
2,1926-07-06,"0,17","0,3","-0,39","0,009"
3,1926-07-07,"0,09","-0,58","0,02","0,009"
4,1926-07-08,"0,21","-0,38","0,19","0,009"


In [104]:
momentum_daily = pd.read_csv("momentum_daily.csv", delimiter=";", header=13)
momentum_daily.fillna(0, inplace=True)
momentum_daily = momentum_daily.iloc[:, :-2]

# Convert date column to timestamp format
momentum_daily = momentum_daily.rename(columns={momentum_daily.columns[0]: 'timestamp'})
momentum_daily = momentum_daily.iloc[:-2]
momentum_daily['timestamp'] = pd.to_datetime(momentum_daily['timestamp'])

On calcule les rentabilités journalières de toutes les cryptos.

In [111]:
#on veut les renta journaliere des differentes crypto et on enlve la premiere valeur car elle est a nan
renta_crypto = crypto_prices.pct_change()
renta_crypto = renta_crypto.dropna()
variance_15d = renta_crypto.resample('15D').std()




on calcule les renta sur 15 jours

In [118]:
crypto_prices_15d = crypto_prices.resample('15D').last()
renta_crypto_15d = crypto_prices_15d.pct_change()
renta_crypto_15d = renta_crypto_15d.dropna()
renta_crypto_15d.head()


,unfi-busd,iota-usd,bhp-usdt,dot-usdt,aave-usdt,nmc-usdt,mx-usdt,dht-usdt,crv-usd,pha-usdt,...,api3-usdt,1inch-usdt,comp-usdt,ersdl-usdt,xor-usdt,ren-usdt,bid-usdt,mkr-usdt,prq-usdt,mta-usdt
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-16,0.499045,0.015430,-0.032069,0.199970,0.925411,0.102759,0.719590,1.132530,1.493763,0.958755,...,1.358723,2.040072,0.555995,0.519978,0.404545,0.128162,0.011737,-0.021507,0.294061,1.012917
2021-01-31,1.738976,1.901137,0.043485,0.696569,0.631100,1.584930,1.103500,0.251059,0.260752,0.422505,...,0.239328,0.340499,0.543869,1.287195,0.596690,0.609369,1.867200,0.736523,0.004158,0.269239
2021-02-15,0.735231,-0.023123,-0.002595,0.229644,-0.227123,-0.304753,0.159469,-0.069150,-0.391548,0.805314,...,-0.177560,-0.265631,-0.038254,1.794321,0.588361,0.206104,-0.477198,-0.146223,0.061117,-0.464631
2021-03-02,-0.244783,0.107405,0.011172,-0.006145,-0.018202,0.014960,-0.059838,0.266250,0.289945,0.319987,...,0.078765,0.063810,-0.082305,3.090701,0.169823,-0.041471,0.095984,-0.070459,0.010659,0.142023
2021-03-17,-0.062108,0.116870,-0.068412,0.055218,0.038643,0.145577,-0.011203,-0.061491,0.279276,0.045428,...,0.562428,0.011361,-0.086347,0.102727,0.066200,-0.053518,0.742541,0.015534,0.385131,0.284588


On calcule pour chaque pair, chaque la difference entre la renta trouve et le rendement sans risque.

In [101]:
# Merge renta_crypto with data_factors on timestamp column
df = renta_crypto.merge(data_factors[['timestamp', 'RF']], on='timestamp')
#on convertit RF en float
df['RF'] = df['RF'].str.replace(',', '.').astype(float)

for column in renta_crypto.columns:
    if column != 'timestamp' and column != 'RF':
        df[column] = df[column] - df['RF']


On calcule sur 15 jours le rendement moyen journalier.

In [100]:
renta_moyenne = renta_crypto.resample('15D').mean()

On obtient donc la valeur du ratio de sharpe correspondant pour chaque crypto sur 15 jours.

In [99]:
ratio_sharp = renta_moyenne.div(variance_15d)

On implemente GetPortfolioCompo

In [105]:
def GetPortfolioCompo(portfolio_index, market_data, rebalancing_dates, returns_horizon):
    portfolio_compo = []
    
    for rebalancing_date in rebalancing_dates:
        # Compute returns St - St-h
        returns = market_data.loc[rebalancing_date] - market_data.loc[rebalancing_date - returns_horizon]
        
        # Order cryptocurrencies in decreasing order of returns
        ordered_cryptos = returns.sort_values(ascending=False)
        
        # Extract cryptocurrencies for portfolio i + 1
        start_index = portfolio_index * 10
        end_index = (portfolio_index + 1) * 10
        portfolio_cryptos = ordered_cryptos.index[start_index:end_index]
        
        portfolio_compo.append(portfolio_cryptos)
    
    return portfolio_compo


In [119]:
import pandas as pd
def UpdateValue(t):
    
    pass

def UpdateCompo(κt, Vt):
    # Implement the logic to update the composition of the portfolio at a rebalancing date
    # κt is the composition at date t
    # Vt is the value of the portfolio at date t
    # Return the updated composition
    pass

def PortfolioBacktest(V, compositions, market_data, rebalancing_dates):
    portfolio_values = []
    
    for t in date_tout_les_15j:
        Vt = UpdateValue(t)
        if t in rebalancing_dates:
            κt = compositions[t]
            κt = UpdateCompo(κt, Vt)
        portfolio_values.append(Vt)
    
    return portfolio_values

# Call the PortfolioBacktest function with the required inputs
V = 1000000  # Initial value of the portfolio
compositions = [...]  # Sequence of compositions
market_data = [...]  # Market data for 50 cryptocurrencies
date_tout_les_15j = crypto_prices.resample('15D').last().index
print(rebalancing_dates)

portfolio_values = PortfolioBacktest(V, compositions, crypto_prices, rebalancing_dates)


DatetimeIndex(['2021-01-01', '2021-01-16', '2021-01-31', '2021-02-15',
               '2021-03-02', '2021-03-17', '2021-04-01', '2021-04-16',
               '2021-05-01', '2021-05-16', '2021-05-31', '2021-06-15',
               '2021-06-30', '2021-07-15', '2021-07-30', '2021-08-14',
               '2021-08-29', '2021-09-13', '2021-09-28', '2021-10-13',
               '2021-10-28', '2021-11-12', '2021-11-27', '2021-12-12',
               '2021-12-27', '2022-01-11', '2022-01-26', '2022-02-10',
               '2022-02-25', '2022-03-12', '2022-03-27', '2022-04-11',
               '2022-04-26', '2022-05-11', '2022-05-26', '2022-06-10',
               '2022-06-25', '2022-07-10', '2022-07-25', '2022-08-09',
               '2022-08-24', '2022-09-08', '2022-09-23', '2022-10-08',
               '2022-10-23', '2022-11-07', '2022-11-22', '2022-12-07',
               '2022-12-22'],
              dtype='datetime64[ns]', name='timestamp', freq='15D')
